# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import linregress
import requests
import time
import datetime
import json

# Import API key
from api_keys import geoapify_key
from citipy import citipy

import warnings
warnings.filterwarnings("ignore")

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ishigaki,24.3448,124.1572,80.73,100.0,75.0,13.80,JP,1716817509
1,1,college,64.8569,-147.8028,47.80,84.0,75.0,0.00,US,1716817727
2,2,banjarmasin,-3.3244,114.5910,78.94,97.0,100.0,3.58,ID,1716817728
3,3,isla vista,34.4133,-119.8610,56.82,80.0,100.0,0.00,US,1716817729
4,4,port mathurin,-19.6833,63.4167,78.03,89.0,18.0,13.47,MU,1716817731


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City"
)

city_data_df = city_data_df.dropna(how="any")


# Display the map
# YOUR CODE HERE

map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
city_data_df.count()

City_ID       553
City          553
Lat           553
Lng           553
Max Temp      553
Humidity      553
Cloudiness    553
Wind Speed    553
Country       553
Date          553
dtype: int64

In [31]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
mask_temp = ((city_data_df['Max Temp'] < 70) & (city_data_df['Max Temp'] > 50))

temp_city_data_df = city_data_df.loc[mask_temp].reset_index()



# Drop any rows with null values
# YOUR CODE HERE
temp_city_data_df.dropna(how="any")

# Display sample data
# YOUR CODE HERE

temp_city_data_df.head()

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,3,3,isla vista,34.4133,-119.8610,56.82,80.0,100.0,0.00,US,1716817729
1,5,5,edinburgh of the seven seas,-37.0676,-12.3116,56.21,77.0,100.0,14.52,SH,1716817732
2,13,13,santa ana,33.7456,-117.8678,59.23,88.0,100.0,5.75,US,1716817679
3,16,17,adamstown,-25.0660,-130.1015,69.15,71.0,45.0,11.03,PN,1716817745
4,17,18,alkmaar,52.6317,4.7486,65.97,53.0,100.0,4.00,NL,1716817746


### Step 3: Create a new DataFrame called `hotel_df`.

In [32]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

hotel_df = temp_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

hotel_df['Hotel Name'] = ""

# Display sample data
# YOUR CODE HERE

hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,isla vista,US,34.4133,-119.8610,80.0,
1,edinburgh of the seven seas,SH,-37.0676,-12.3116,77.0,
2,santa ana,US,33.7456,-117.8678,88.0,
3,adamstown,PN,-25.0660,-130.1015,71.0,
4,alkmaar,NL,52.6317,4.7486,53.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 # YOUR CODE HERE

categories = "accommodation.hotel"
coonditions = "internet_access"
limit = 1
# filters = f"circle:{longitude},{latitude},{radius}"
# bias = f"proximity:{longitude},{latitude}"

params = {
    "categories":categories,
    "conditions":conditions,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key    
}
    # YOUR CODE HERE
   

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    longitude = hotel_df["Lng"]
    Latitude = hotel_df['Lat']
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"# YOUR CODE HERE
    params["bias"] = f"proximity:{longitude},{latitude}"# YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = name_address.json() # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Hotel del Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)